In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! apt -y install gh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gh is already the newest version (2.4.0+dfsg1-2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
! git clone -b responses https://github.com/lightblue-tech/M-IFEval.git

Cloning into 'M-IFEval'...
remote: Enumerating objects: 641, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 641 (delta 71), reused 312 (delta 56), pack-reused 304 (from 1)
Receiving objects: 100% (641/641), 29.55 MiB | 15.89 MiB/s, done.
Resolving deltas: 100% (259/259), done.


# Response generation

In [ ]:
! cd M-IFEval && pip install -q -r requirements.txt
! pip install -q vllm bitsandbytes hf-transfer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from google.colab import userdata

! huggingface-cli login --token {userdata.get('HF_TOKEN')}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
local_model_names = ['CohereForAI/c4ai-command-r-plus-4bit',
 'CohereForAI/c4ai-command-r-v01-4bit',
 'CohereForAI/aya-23-8B',
 'Qwen/Qwen2.5-0.5B-Instruct-GPTQ-Int4',
 'Qwen/Qwen2.5-1.5B-Instruct-GPTQ-Int4',
 'Qwen/Qwen2.5-3B-Instruct-GPTQ-Int4',
 'Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4',
 'Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4',
 'Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4',
 'Qwen/Qwen2.5-72B-Instruct-GPTQ-Int4',
 'hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4',
 'hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4',
 'mistralai/Mistral-7B-Instruct-v0.3']

for local_model_name in local_model_names:
    ! cd M-IFEval && HF_HUB_ENABLE_HF_TRANSFER=1 python get_responses.py --model_name {local_model_name}

2024-10-02 07:05:25.753839: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 07:05:25.770364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 07:05:25.791711: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 07:05:25.798098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 07:05:25.813480: I tensorflow/core/platform/cpu_feature_guar

# Evaluation

In [ ]:
! cd M-IFEval && pip install -q -r requirements.txt

/bin/bash: line 1: cd: M-IFEval: No such file or directory


In [ ]:
from glob import glob

input_paths = glob("./data/*_input_data.jsonl")

for input_path in input_paths:
    response_paths = glob(input_path[:-10] + "response_data_*")

    for response_path in response_paths:
        run_name = response_path.split("/")[-1].split(".")[0]

        ! mkdir -p ./evaluations/{run_name}
        ! python -m evaluation_main \
          --input_data={input_path} \
          --input_response_data={response_path} \
          --output_dir=./evaluations/{run_name}


# Visualisation

In [ ]:
%cd M-IFEval
# ! pip install -q -r requirements.txt

/content/M-IFEval


In [ ]:
from glob import glob
import pandas as pd
from tqdm.auto import tqdm

paths = glob("./data/*_input_data.jsonl")

model_results = []

for path in tqdm(paths):
    lang_name = path.split("/")[-1].split("_")[0]
    res_df = pd.read_json(path, lines=True)

    print(lang_name)
    print(res_df.shape)
    print(res_df.instruction_id_list.str.len().value_counts())
    print(res_df.instruction_id_list.explode().value_counts().min())

    # exploded_res_df["model_name"] = model_name[len("es_input_response_data_"):]
    # exploded_res_df["language"] = model_name[:2]

    # model_results.append(exploded_res_df)

  0%|          | 0/4 [00:00<?, ?it/s]

es
(115, 4)
instruction_id_list
1    100
2      8
3      7
Name: count, dtype: int64
4
ja
(172, 4)
instruction_id_list
1    128
2     34
3     10
Name: count, dtype: int64
4
fr
(235, 4)
instruction_id_list
1    146
2     68
3     21
Name: count, dtype: int64
7
en
(541, 4)
instruction_id_list
1    305
2    179
3     57
Name: count, dtype: int64
10


In [ ]:
from glob import glob
import pandas as pd
from tqdm.auto import tqdm

paths = glob("./evaluations/*/eval_results_strict.jsonl")

select_models = {
    'gpt-4o-2024-08-06': "GPT4o",
    'o1-mini-2024-09-12': "o1 Mini",
    'claude-3-5-sonnet-20240620': "Sonnet",
    'Qwen__Qwen2.5-32B-Instruct-GPTQ-Int4': "Qwen 2.5 32B I.",
    'gpt-4o-mini-2024-07-18': "GPT4o Mini",
    'claude-3-haiku-20240307': "Haiku",
    'claude-3-opus-20240229': "Opus",
    'o1-preview-2024-09-12': "o1"
}

model_results = []

for path in tqdm(paths):
    run_name = path.split("/")[-2]
    model_name = run_name[len("es_input_response_data_"):]
    if model_name not in select_models:
        continue
    res_df = pd.read_json(path, lines=True)
    res_df["instr_len"] = res_df.follow_instruction_list.str.len()

    exploded_res_df = pd.DataFrame(res_df.apply(
        lambda x: [
            {
                "instruction_id": inst, "follow_bool": val
            } for inst, val in zip(x["instruction_id_list"], x["follow_instruction_list"])
        ],
        axis=1).explode())

    exploded_res_df["instruction_id"] = exploded_res_df[0].apply(lambda x: x["instruction_id"])
    exploded_res_df["follow_bool"] = exploded_res_df[0].apply(lambda x: x["follow_bool"])
    exploded_res_df = exploded_res_df.drop(0, axis=1)
    exploded_res_df["model_name"] = select_models[model_name]
    exploded_res_df["language"] = run_name[:2]
    idx_val_counts = exploded_res_df.index.value_counts()
    exploded_res_df["num_instr"] = exploded_res_df.index.map(idx_val_counts)

    model_results.append(exploded_res_df)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
full_results_df = pd.concat(model_results)
full_results_df["instruction_stem"] = full_results_df.instruction_id.str.split(":").str[1:3].str.join(":")

### Per model analysis

In [ ]:
grouped_df = full_results_df.groupby(
    ["model_name", "language"]
    ).follow_bool.mean().reset_index(drop=False).pivot(
        index="model_name", columns="language", values="follow_bool"
        )

In [ ]:
grouped_df["mean_esfrja"] = grouped_df[["es", "fr", "ja"]].mean(axis=1)

In [ ]:
grouped_df.sort_values("mean_esfrja", ascending=False).round(3) * 100

language,en,es,fr,ja,mean_esfrja
model_name,,,,,
o1,86.7,92.7,91.3,75.7,86.6
Opus,87.3,90.5,87.0,75.7,84.4
Sonnet,88.1,87.6,88.1,77.0,84.2
o1 Mini,83.9,92.0,88.4,69.5,83.3
GPT4o,88.6,89.8,87.8,70.4,82.7
GPT4o Mini,86.0,85.4,85.5,65.9,78.9
Qwen 2.5 32B I.,86.0,82.5,81.7,65.9,76.7
Haiku,77.3,78.8,78.3,61.9,73.0


### Per model (only unique categories)

In [ ]:
unique_instruction_list = ['detectable_content:informal_address',
 'detectable_content:no_digits',
 'detectable_format:nominal_ending',
 'detectable_format:number_numbered_lists',
 'length_constraints:number_letters',
 'letters:furigana',
 'letters:hiragana_only',
 'letters:kanji',
 'letters:kansuuji',
 'letters:katakana_only',
 'letters:no_hiragana',
 'letters:no_katakana',
 'punctuation:exclamation_marks',
 'punctuation:no_period',
 'punctuation:question_marks',
 'special_character:accents',
 'special_character:dieresis',
 'special_character:enie',
 'special_character:ethel_or_cedilla',
 'special_character:no_accents',
 'special_character:tildes',
 'startend:sentence_unified_end']

grouped_df = full_results_df[full_results_df.instruction_stem.isin(unique_instruction_list)].groupby(
    ["model_name", "language"]
    ).follow_bool.mean().reset_index(drop=False).pivot(
        index="model_name", columns="language", values="follow_bool"
        )

grouped_df["mean_esfrja"] = grouped_df[["es", "fr", "ja"]].mean(axis=1)

In [ ]:
grouped_df.sort_values("mean_esfrja", ascending=False).round(3) * 100

language,es,fr,ja,mean_esfrja
model_name,,,,
o1,75.0,96.1,61.4,77.5
Sonnet,66.7,90.2,70.5,75.8
Opus,62.5,90.2,64.8,72.5
GPT4o,58.3,80.4,55.7,64.8
o1 Mini,66.7,72.5,50.0,63.1
Qwen 2.5 32B I.,54.2,78.4,54.5,62.4
Haiku,54.2,80.4,52.3,62.3
GPT4o Mini,58.3,68.6,47.7,58.2


### Per model (only non-unique)

In [ ]:
grouped_df = full_results_df[~full_results_df.instruction_stem.isin(unique_instruction_list)].groupby(
    ["model_name", "language"]
    ).follow_bool.mean().reset_index(drop=False).pivot(
        index="model_name", columns="language", values="follow_bool"
        )

grouped_df["mean_esfrja"] = grouped_df[["es", "fr", "ja"]].mean(axis=1)

In [ ]:
grouped_df.sort_values("mean_esfrja", ascending=False).round(3) * 100

language,en,es,fr,ja,mean_esfrja
model_name,,,,,
o1,86.7,96.5,90.5,84.8,90.6
o1 Mini,83.9,97.3,91.2,81.9,90.1
Opus,87.3,96.5,86.4,82.6,88.5
GPT4o,88.6,96.5,89.1,79.7,88.4
Sonnet,88.1,92.0,87.8,81.2,87.0
GPT4o Mini,86.0,91.2,88.4,77.5,85.7
Qwen 2.5 32B I.,86.0,88.5,82.3,73.2,81.3
Haiku,77.3,84.1,77.9,68.1,76.7


### Per instruction results

In [ ]:
grouped_df = full_results_df.groupby(
    ["model_name", "instruction_stem"]
    ).follow_bool.mean().reset_index(drop=False).pivot(
        index="model_name", columns="instruction_stem", values="follow_bool"
        )

In [ ]:
lang_map = full_results_df.groupby(["instruction_stem"]).language.apply(set)

per_instr_scores = grouped_df.T.mean(axis=1).sort_index().reset_index(drop=False).join(
    lang_map.sort_index().reset_index(drop=True)
    ).sort_values(0, ascending=True)

In [ ]:
per_instr_scores["instruction_family"] = per_instr_scores["instruction_stem"].str.split(":").str[0].str.replace("_", " ").str.capitalize()
per_instr_scores["instruction_name"] = per_instr_scores["instruction_stem"].str.split(":").str[1].str.replace("_", " ").str.capitalize()
per_instr_scores["language"] = per_instr_scores["language"].apply(lambda x: ", ".join([y.upper() for y in x]))

per_instr_scores[["instruction_family", "instruction_name", "language", 0]]

,instruction_family,instruction_name,language,0
44,Special character,Enie,ES,0.000000
37,Letters,No katakana,JA,0.142857
31,Letters,Furigana,JA,0.145833
43,Special character,Dieresis,ES,0.156250
50,Startend,Sentence unified end,JA,0.339286
36,Letters,No hiragana,JA,0.357143
32,Letters,Hiragana only,JA,0.482143
45,Special character,Ethel or cedilla,FR,0.602273
35,Letters,Katakana only,JA,0.604167
46,Special character,No accents,FR,0.637500
